# Stonks!
## A realtime stocks analyzer.

Coded by **Alfio Cardillo**
<br>
For the subject **"Technologies for Advanced Programming"**
<br>
University of **Catania**, Department of Mathematics and Computer Science. **13 June 2023**

# What is it?
This project consists in a realtime stocks analyzer of the three most valued stocks in the market right now. (June 2023). Which are Apple (**AAPL**), Alphabet Inc. (**GOOGL**) and Microsoft (**MSFT**).
<br>

<div>
<img src="images/logos.png" width="800"/>
</div>
   

# The structure
The pipeline consists in a variety of technologies that are all dockerized into **small micro-services** that communicates between one another thanks to **docker networks**.
<br><br>
Some time was spent in optimizing the containers by both choosing when possible an **official image** for that technology and the **smallest version**.

<div>
<img src="images/pipeline.png" width="1000"/>
</div>

# Script
A python script downloads the data from [Alpha Vantage](https://www.alphavantage.co/) for the three symbols, then internally it creates a **timeline** and arranges all events on it based on the time they have in the record. 
<br><br>
Lastly it creates an **internal timer** and when the time arrives at a point in the timeline where an event happened it sends that event to Fluentbit with an HTTP request on Fluentbit's port **9880**.

## Why AlphaVantage and Python?
- It is **hard to find realtime data** in the stock market that are **free to use**
- Easy language that gives especially for these low resource demanding task an easy and great solution

# Fluentbit
Fluentbit receives the data from python and **parse** it by extracting the **event time** and converting it into a **Unix Timestamp**
<div>
<img src="images/parse.png" width="500"/>
</div>

# Why Fluentbit
- **Smaller** and **faster** than other ingestion methods
- Still an easy syntax 
<div>
<img src="images/fluentlogstash.jpg" width="300"/>
</div>

# Difficulties
- The **documentation** is **not very clear** on how to parse data

# Kafka
<div>
<img src="images/nozookeeper.gif" width="100"/>
</div>



# Spark

# Elastic Search


## Why
<div>
<img src="images/kibanaelastic.jpg", width="300">
</div>

# Kibana


## Why
- Ease of use